# VPR specific QUBO

### Constraint on edges for all of the nets:

$\sum_{i \in \{edges\}}\sum_{p, q \in \{nets\} p \neq q} \sum_{r \in p, s \in q} y_{p,r,i}y_{q,s,i}$

### Constraint on edges for the source node on each rout of each net: 

$(1-\sum_i y_{p,r,i} + \sum_{i\neq j} 2y_{p,r,i}y_{p,r,j})$

### Constraint on wires (CHANX CHANY) to make sure they have one input and as many outputs as inputs:

$\sum_{i,j \in input, i \neq j} x_{p,r,i}x_{p,r,j} + (\sum_{i, x \in input x_{p,r,i}} - \sum_{j, y \in output} y_{p,r,j} )^2$

### The objective function and heuristic for coeficient are from the QUBO_toy_problem_old 

In [1]:
#import xml.etree.ElementTree as ET
from pathlib import Path
# from qpr.routing import NetList, Route
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)
from  qpr.vtr_utils import parse_rr_graph_xml

from itertools import combinations_with_replacement as cwr
from itertools import product as prd

import re

%matplotlib inline

In [31]:
name = 'sharedgraphstuff/example2'
fpath = Path(name + '.xml')
graph, rr_nodes, _ , _ = parse_rr_graph_xml(fpath)

In [32]:
# This is how we define a net
#nets = {1 : {'SOURCE' : '199','SINK' : ['120']}}

In [33]:
# To read the netlist from the route file
fpath = Path(name +'.route')
with open(fpath, 'r') as f:
    datalines = f.readlines()
net_num = 0
nets = {}
for item in datalines:
    if 'SOURCE' in item:
        net_num += 1
        nets[net_num] = {'SOURCE' : re.findall(r'\d+', item[:item.find('SOURCE')])[0] ,'SINK' : []}
    if 'SINK' in item:
        #print(re.findall(r'\d+', item[:item.find('SINK')]))
        nets[net_num]['SINK'] += (re.findall(r'\d+', item[:item.find('SINK')]))

### Some preprocessing and claen ups

In [34]:
# remove useless nodes perhaps this should be made into a 
degrees = {node:val for (node, val) in graph.degree()}
#graph, rr_nodes = parse_rr_graph_xml(fpath)
graph1 = graph.copy()
SINK_dict = {}
SOURCE_dict = {}
nets_source = []
nets_sink = [] 
for i in rr_nodes.keys():
    if (not(rr_nodes[i]['type'] in ['SOURCE', 'SINK']) and degrees[i] == 1):
        graph1.remove_node(i)
graph2 = graph1.copy()
degrees = {node:val for (node, val) in graph1.degree()}
for i in graph1.nodes:
    if degrees[i] == 0:
        graph2.remove_node(i)
for item in nets.keys():
    nets_source.append(nets[item]['SOURCE'])
    nets_sink += nets[item]['SINK']
    SINK_dict.update(dict(graph2.in_edges(nets[item]['SINK']))) 
    SOURCE_dict.update(dict(graph2.out_edges(nets[item]['SOURCE'])))
#remove Source/Sinks that are not in the net
graph3 = graph2.copy()
for i in graph2.nodes:
    if rr_nodes[i]['type'] == 'SOURCE' and not (i in nets_source):
        graph3.remove_node(i)
    elif rr_nodes[i]['type'] == 'SINK' and not (i in nets_sink):
        graph3.remove_node(i)
    elif rr_nodes[i]['type'] == 'IPIN' and not(i in SINK_dict.keys()):
        graph3.remove_node(i)
    elif rr_nodes[i]['type'] == 'OPIN' and not(i in SOURCE_dict.values()):
        graph3.remove_node(i)

In [35]:
#nets

In [36]:
len(graph3)

2907

In [37]:
def VPR2QUBO(G, nets, rr_nodes):
    Q = {}
    relevant_edge_list = []
    for net in nets.keys():
        #Constraint on SOURCE    
        for r, _ in enumerate(nets[net]['SINK']):
            pair_list = []
            for item in G.out_edges(nets[net]['SOURCE']):
                if not (item in relevant_edge_list):
                    relevant_edge_list.append(item)
                pair_list.append(f'Net_{net}_Edge_{item}_Rout_{r}')
            for i, j in cwr(pair_list, 2):
                if i == j:
                    Q[(i, j)] = -1
                else:
                    Q[(i, j)] = 2
        #Constraint on SINK
        for r, sink_node in enumerate(nets[net]['SINK']):
            pair_list = []
            for item in G.in_edges(sink_node):
                if not (item in relevant_edge_list):
                    relevant_edge_list.append(item)
                pair_list.append(f'Net_{net}_Edge_{item}_Rout_{r}')
            for i, j in cwr(pair_list, 2):
                if i == j:
                    Q[(i, j)] = -1
                else:
                    Q[(i, j)] = 2
        #Constraint on Wires
        for r, sinks in enumerate(nets[net]['SINK']):
            distance = nx.dijkstra_path_length(graph.subgraph(graph3.nodes), nets[net]['SOURCE'], sinks)
            for item in G.nodes:
                pair_list_in = []
                pair_list_out = []
                if not (rr_nodes[item]['type'] in ['SINK', 'SOURCE', 'IPIN', 'OPIN']):
                    for i in G.in_edges(item):
                        if not (item in relevant_edge_list):
                            relevant_edge_list.append(item)  
                        pair_list_in.append(f'Net_{net}_Edge_{i}_Rout_{r}')
                    for i in G.out_edges(item):
                        if not (item in relevant_edge_list):
                            relevant_edge_list.append(item) 
                        pair_list_out.append(f'Net_{net}_Edge_{i}_Rout_{r}')
                    #making sure that there is only one input to the node
                    for i, j in cwr(pair_list_in, 2):
                        if i != j:
                            Q[(i, j)] = 1
                    #making sure that there are as many inputs as outputs
                    for i, j in cwr(pair_list_in, 2):
                        if i == j:
                            Q[(i, j)] = 1
                        else:
                            Q[(i, j)] = 2
                    for i, j in cwr(pair_list_out, 2):
                        if i == j:
                            Q[(i, j)] = 1
                        else:
                            Q[(i, j)] = 2
                    for i in pair_list_in:
                        for j in pair_list_out:
                            Q[(i, j)] = -2
        #Objective function: trying to minimize the number of edges(switches)
                    for i in pair_list_in:
                        Q[(i, i)] += 1 / distance
                    for i in pair_list_out:
                        Q[(i, i)] += 1 / distance
# constraints on the edges of various nets
    for i in relevant_edge_list:
        for p, q in cwr(nets.keys(), 2):
            if p != q:
                for r, _ in enumerate(nets[p]['SINK']):
                    for s, _ in enumerate(nets[q]['SINK']):
                        Q[(f'Net_{net}_Edge_{p}_Rout_{r}', f'Net_{net}_Edge_{q}_Rout_{s}')] = 1
    return Q

In [38]:
Q = VPR2QUBO(graph.subgraph(graph3.nodes), nets, rr_nodes)

In [39]:
len(Q)

57254377

In [40]:
new_Q = {}
code = {} # {QUBO_Node : new node}
i = 0
for item in Q.keys():
    if not (item[0] in code.keys()):
        i += 1
        code[item[0]] = i
    if not (item[1] in code.keys()):
        i += 1
        code[item[1]] = i
    new_Q[(code[item[0]], code[item[1]])] = Q[(item[0], item[1])]

In [41]:
fpath = Path(name +'.pickle')
with open(fpath, 'wb') as f:
    pickle.dump(new_Q, f, protocol=pickle.HIGHEST_PROTOCOL)
fpath = Path(name +'_code.pickle')
with open(fpath, 'wb') as f:
    pickle.dump(code, f, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
len(code) # this is the number of variable also

4987548